# Matmul

$
\begin{align*}
\overset{ (N,M) }{ Y } 
&= 
\begin{bmatrix}
{ Y_{(n=0)} } \\
\vdots \\
{ Y_{(n)} } \\
\vdots \\
{ Y_{(n=N-1)} }
\end{bmatrix}
=
\overset{ (N,D) }{ X } \; @ \; \overset{ (D,M) }{ W^T }
\\
\\
\overset{ (M,) }{ Y_{(n)} } &= (y_{(n)(m=0)}, \; \dots, \; y_{(n)(m)},  \; \dots, \; y_{(n)(m=M-1)})
\\
\overset{ () }{ y_{(n)(m)} }
&= \overset{ (D,) }{ X_{(n)} } \cdot \overset{ (D,) }{ W_{(m)}^T }
= \sum\limits ^{D}_{d=0}  \overset{ () }{ x_{(n)(d)} } * \overset{ () }{ w_{(m)(d)} }
\\
_{(0 \le d \le D, \; 0 \le n \lt N, \; 0 \le m \lt M)}
\end{align*}
$

# Activation

$
\begin{align*}
\overset{ (N,M) }{ A } &= 
activation \left( 
    \overset{ (N,M) }{ Y }  = 
    \begin{bmatrix}
    { Y_{(n=0)} } \\
    \vdots \\
    { Y_{(n)} } \\
    \vdots \\
    { Y_{(n=N-1)} }
    \end{bmatrix}
\right)
\\
\overset{ (M,) }{ A_{(n)} } 
&= activation \left( \overset{ (M,) }{ Y_{(n) }} \right)  \\
&= (a_{(n)(m=0)}, \; \dots, \; a_{(n)(m)},  \; \dots, \; a_{(n)(m=M-1)})
\\
\overset{ () }{ a_{(n)(m)} } &= activation \left( \overset{ () }{ y_{(n)(m)} } \right)
\quad _{(0 \le n \lt N, \; 0 \le m \lt M)}
\end{align*}
$

# Softmax
$C_n$ is to prevent the overflow at $np.exp()$.

```
def softmax(x):
    """Softmax function from deep-learning-from-scratch
    Args:
        x: batch input data of shape (N x M).
            N: Batch size
            M: Number of nodes
    Returns:
        Prediction probability matrix P of shape (N x M)
    """
    c = np.max(x, axis=-1, keepdims=True)   # オーバーフロー対策
    exp = np.exp(x - c)
    return exp / np.sum(exp, axis=-1, keepdims=True)
```

<img src="images/SMLL.png" align="left" />

$
\begin{align*}
\overset{ (N,1) }{ C } &= np.max\left( 
    \overset{ (N,M) }{ A }, \; axis=-1,  \; keepdim=True 
\right)
\\
\overset{ (N,M) }{ EXP } &= np.exp \left( \overset{ (N,M) }{ A } - \overset{ (N,1) }{ C } \right)
= np.exp \left(
    \begin{bmatrix}
    { A_{(n=0)} } - { C_{(n=0)} }\\
    \vdots \\
    { A_{(n)} }   - { C_{(n)} }\\
    \vdots \\
    { A_{(n=N-1)} } - { C_{(n=N-1)} }\\
    \end{bmatrix}
\right) 
\\
\overset{ (N,1) }{ sum(EXP) } &= np.sum \left( 
    \overset{ (N,M) }{ EXP }, \; axis=-1,  \; keepdim=True 
\right)
\\
\overset{ (N,M) }{ P } &= \overset{ (N,M) }{ EXP }  \;\; / \;\; \overset{ (N,1) }{ sum(EXP) } 
\\
\overset{ (N,) }{ P_{(n)} } &= (p_{(n)(m=0)}, \; \dots, \; p_{(n)(m)} , \; \dots, \; p_{(n)(m=M-1)})
\\
{ p_{(n)(m)} } 
&= \frac {np.exp \left( 
    { a_{(n)(m) } } - { c_{(n)(m)} }) \right) 
}
{  
np.sum \left( 
    np.exp \left( 
        a_{(n)(m) } - c_{(n)(m)}
    \right)
\right) 
}
\end{align*}
$

# Cross Entropy Log Loss

$
LP = label\_probability = P \left[ \\
\quad ( \; 0, \; \dots, \;  {N-1}) , \\
\quad ( \; t_{(n=0)} \; , \dots , \; t_{(n=N-1)}) \\
\right]
\\
\begin{align*}
\overset{ (N,) }{ J } &= np.sum( np.log(LP), axis = -1) \\
\text{ for index labels e.g. (5, 2, 0, 9, ...)}
\\
\overset{ (N,) }{ J } &= \sum\limits^{M-1}_{m=0} 
    \left[ \; \;  
        t_{(n)(m)} \;  * \;  np.log(p_{(n)(m)}) \;\;  
    \right]
\\
\text{ for one hot encoding labels }
\\
\overset{ () }{ L } &= \frac {1}{N} \sum\limits^{N-1}_{n=0} \overset{ () }{ J_{{(n)}} }
\\
\overset{ () }{ L } &= \frac {1}{N} \sum\limits^{N-1}_{n=0} \sum\limits^{M-1}_{m=0} 
    \left[ \; \;  
        t_{(n)(m)} \;  * \;  np.log(p_{(n)(m)}) \;\;  
    \right]
\\
\sum\limits^{M-1}_{m=0}
\\
\frac {1}{N} \sum\limits^{N-1}_{n=0} 
\end{align*}
$

$\begin{align*}
\overset{ (N,M) }{ T } &= ( 
    \overset{ (N,) }{ T_{(0)} }, \; \dots \;, \overset{ (N,) }{ T_{(N-1)} } 
) \qquad \text { for one hot encoding labels} 
\\
\overset{ (M,) }{ T_{(n)} } &= ( \overset{ () }{ t_{(n)(0)} }, \; \dots \;, \overset{ () }{ t_{(n)(M-1)} })
\\
\overset{ (N,) }{ T } &= (\overset{ () }{ T_{(0)} }, \; \dots \;, \overset{ () }{ T_{(N-1)} }) \qquad \text {for indexing labels }
\end{align*}
$

In [2]:
import numpy as np

In [12]:
a = np.arange(6).reshape(2,3)
print(a)
b = np.arange(0,-6,-1).reshape(2,3)
print(b.T)
c = [
    np.inner(a[0], b[0]),
    np.inner(a[1], b[1]),    
]
print(c)

[[0 1 2]
 [3 4 5]]
[[ 0 -3]
 [-1 -4]
 [-2 -5]]
[-5, -50]
